# Python Code to make the logic first approach of a truth table

-> A .v (verilog file) is needed

-> We have to create an array/matrix which will represent the truth table

-> The code takes de .v and the matrix and create a .v file with lots of logic elements. Then the size can be reduced by passign it through a syntethizer.

## Function to convert a line of the matrix from integer input to binary output

-> In: Must be a integer which will be the input binary number

-> BitIn: Length of the input

-> Out: Is the output of the truth table

-> BitOut: Length of the output

->strIN: name of the input register (ex: regIN[length-1:0])

->end: if 1 this means that is an end line and whill put a ";" at the end

In [1]:
def convert_to_binary_and_generate_lines(In, BitIn, Out, BitOut, strIN, end):
    """Convertir cada número a binario de 16 bits y generar líneas para bits que son 1."""

    line="" #line to be written
    
    # convert to binary string
    binary_str_Out = format(Out, f"0{BitOut}b")
    binary_str_In = format(In, f"0{BitIn}b")
    
    #print(binary_str_In)
    #print(binary_str_Out)
    

    # Obtener los índices de los bits que son 1 (invirtiendo los índices)
    #ones_indices = [15 - i for i, bit in enumerate(binary_str) if bit == '1']
    #print (number)
    #print(binary_str)
    lineG=""
    for jind,j in enumerate(binary_str_Out):
        if(j=="1"): #Para cada linea que ES 1
            terms=[]
            for ind,bit in enumerate(binary_str_In):
                if bit=="1":
                    term = f"({strIN}[{BitIn - 1 - ind}])"
                else:
                    term = f"(~{strIN}[{BitIn - 1 - ind}])"
                terms.append(term)
            and_expression = "&".join(terms)
            #line = "\t\t\t\t ((" + and_expression + ")<<("+str(BitOut-1-jind)+")) |\n" #we can put as many tabs we want
            line = "\t\t ((" + and_expression + ")<<("+str(BitOut-1-jind)+")) |\n" #we can put as many tabs we want
            lineG = lineG+line
    
    if(end==1):
        lineG=lineG + "\t\t 1'b0 ; \n" #es irrelevante ya que trabajamos con or y se eliminara
    return lineG

#PRUEBA

In = 7
BitIn = 8
Out= 10039
BitOut = 16
strIN="inLine_ld"


a=convert_to_binary_and_generate_lines(In, BitIn, Out, BitOut, strIN, 1)
print(a)

## Function to create the verilog file ##

-> verilog_file : Is the input verilog file in where the truth table will be inserted

-> output_file : The output verilog file

-> strStart : Is the comment line of the input line where the combinational logic need to be inserted

-> data_list[2][N] : 2-dimensional N vector whre the first component are the input values and the second the output

In [2]:
def modify_verilog_file(verilog_file, output_file, strStart, data_list, StrIn, BitIn, BitOut):
    with open(verilog_file, 'r') as infile, open(output_file, 'w') as outfile:
        add_lines = False

        for line in infile:
            # Escribir la línea actual del archivo Verilog
            outfile.write(line)

            # Detectar la línea que contiene `//start python`
            if strStart in line: #linea de start Python
                add_lines = True  # Activar el flag para agregar líneas
                continue

            # Añadir las nuevas líneas después de `//start python`
            if add_lines:
                
                for i in range(0,len(data_list[0])):
                    if(i==(len(data_list[0])-1)):
                        new_line=convert_to_binary_and_generate_lines(data_list[0][i], BitIn, data_list[1][i], BitOut, StrIn, 1)
                    else:
                        new_line=convert_to_binary_and_generate_lines(data_list[0][i], BitIn, data_list[1][i], BitOut, StrIn, 0)
                    outfile.write(new_line)
                add_lines = False  # Desactivar el flag una vez que se agreguen las líneas

## Example

El fitxer d'entrada es diu "verilog_py.v" i el de sortida es dira "log_e.v"

## Funcion a desarrollar (creamos matriz de In / Out)

In [3]:
# la funcion toma un numero entero y calcula el log en base 2 más cercano entero (por debajo)
def fun(x):
    if(x<2): # si es log(0) o log(1) retornem 0
        return 0
    else:
        i=0
        while(x>2**i):
            i+=1
        y=i-1
        return y

In [4]:
# ahora cojemos el programa verilog_py,
# donde tenemos como configuracion:
#   bitIn = 12 i bitOut = 4
#   el señal se llama "numA" y el start se llama "//start python" (hay 4 tabuladores)
verilog_file = "verilog_py.v"
output_file = "log2n_integer.v"
strStart="//start python"
StrIn="numA"
BitIn = 12
BitOut = 4

data_list = [[],[]]

for i in range(0, 2**BitIn):
    data_list[0].append(i)
    data_list[1].append(fun(i))

modify_verilog_file(verilog_file, output_file, strStart, data_list, StrIn, BitIn, BitOut)